In [55]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [56]:
import os
default_path = '/Users/Rex/Desktop'
os.chdir(default_path)

In [57]:
import math
data = pd.read_csv("rex_22.csv")
#處理資料
data_no = data.dropna()

x = data_no.drop('y',1)
y = data_no['y']
x = x.shift(1)

x_new = x.drop(1)
y_new = y.drop(1)

print(x_new[:5])
print(y_new[:5])

   udnew       OI     net  opmcl    short
2   27.0  56979.0 -1121.0   38.0  33765.0
3  -81.0  57267.0 -1545.0  -24.0  32775.0
4  -43.0  57605.0 -3215.0  -32.0  33413.0
5  -55.0  57930.0 -5085.0   -9.0  36407.0
6   53.0  58371.0  1160.0  -23.0  34722.0
2    0
3    0
4    0
5    1
6    1
Name: y, dtype: int64


In [61]:
x_new['OI'] = np.log(x['OI'])
x_new['short'] = np.log(x['short'])
x_new['OI*ud'] = x_new['OI']*x_new['udnew']

x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size = 0.3, random_state=0)

In [62]:
#Standardlisze,但後來發現不標準化，模型較準，故下面套模型時不使用標準化的特徵
sc = StandardScaler()
sc.fit(x_train)
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

#這一段是網路找來的，不確定是否有更好的寫法
tuned_parameters=[{'penalty':['l1','l2'],
                   'C':[0.01,0.05,0.1,0.5,1,5,10,50,100],
                    'solver':['liblinear'],
                    'multi_class':['ovr']},
                {'penalty':['l2'],
                 'C':[0.01,0.05,0.1,0.5,1,5,10,50,100],
                'solver':['lbfgs'],
                'multi_class':['ovr','multinomial']}]
LGGR=GridSearchCV(LogisticRegression(tol=1e-6),tuned_parameters,cv = 10)
LGGR.fit(x_train,y_train)



GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'penalty': ['l1', 'l2'], 'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100], 'solver': ['liblinear'], 'multi_class': ['ovr']}, {'penalty': ['l2'], 'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100], 'solver': ['lbfgs'], 'multi_class': ['ovr', 'multinomial']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [63]:
y_pred = LGGR.predict(x_test)
y_pred_prob = LGGR.predict_proba(x_test)

from sklearn import metrics
confusion = metrics.confusion_matrix(y_test,y_pred)

print("accuracy_score: "+str(metrics.accuracy_score(y_test,y_pred)))
print(confusion)
print("f1_score: "+str(metrics.f1_score(y_test,y_pred)))
print("R-squre: "+str(metrics.r2_score(y_test,y_pred)))

accuracy_score: 0.5675675675675675
[[16 60]
 [ 4 68]]
f1_score: 0.6799999999999999
R-squre: -0.7309941520467838
